In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path 
import missingno as msno
from tabulate import tabulate
from statistics import median, mean, quantiles
import pprint
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from datetime import datetime

In [2]:
path = Path(os.getcwd()) 
p_parent = path.parent
p_customer = str(p_parent) + '\data\olist_customers_dataset.csv'
p_geo = str(p_parent) + '\data\olist_geolocation_dataset.csv'
p_order_item = str(p_parent) + '\data\olist_order_items_dataset.csv'
p_order_payment = str(p_parent) + '\data\olist_order_payments_dataset.csv'
p_order_review = str(p_parent) + '\data\olist_order_reviews_dataset.csv'
p_orders = str(p_parent) + '\data\olist_orders_dataset.csv'
p_products = str(p_parent) + '\data\olist_products_dataset.csv'
p_sellers = str(p_parent) + '\data\olist_sellers_dataset.csv'
p_category = str(p_parent) + '\data\product_category_name_translation.csv'

customer = pd.read_csv(p_customer)
geo = pd.read_csv(p_geo)
order_item = pd.read_csv(p_order_item)
order_payment = pd.read_csv(p_order_payment)
order_review = pd.read_csv(p_order_review)
orders = pd.read_csv(p_orders)
products = pd.read_csv(p_products)
sellers = pd.read_csv(p_sellers)
category = pd.read_csv(p_category)

In [11]:
df = orders.merge(customer, on="customer_id", how ='left')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
 8   customer_unique_id             99441 non-null  object
 9   customer_zip_code_prefix       99441 non-null  int64 
 10  customer_city                  99441 non-null  object
 11  customer_state                 99441 non-null  object
dtypes: int64(1), object(11)
memory usage: 9.9+ MB


In [5]:
output = []
for col in df.columns:

    nonNull  = len(df) - np.sum(pd.isna(df[col]))
    nonNullprop = (nonNull / len(df[col]))*100
    unique = df[col].nunique()
    colType = str(df[col].dtype)
    output.append([col, colType, nonNull, round(nonNullprop, 1) , unique])
        

df_stats = pd.DataFrame(output)
df_stats.columns = ['nom colonne','dtype', 'valeur non null',"% de non null", 'nb_unique']

print(tabulate(df_stats, headers='keys', tablefmt='psql'))

+----+-------------------------------+---------+-------------------+-----------------+-------------+
|    | nom colonne                   | dtype   |   valeur non null |   % de non null |   nb_unique |
|----+-------------------------------+---------+-------------------+-----------------+-------------|
|  0 | order_id                      | object  |             99441 |           100   |       99441 |
|  1 | customer_id                   | object  |             99441 |           100   |       99441 |
|  2 | order_status                  | object  |             99441 |           100   |           8 |
|  3 | order_purchase_timestamp      | object  |             99441 |           100   |       98875 |
|  4 | order_approved_at             | object  |             99281 |            99.8 |       90733 |
|  5 | order_delivered_carrier_date  | object  |             97658 |            98.2 |       81018 |
|  6 | order_delivered_customer_date | object  |             96476 |            97   |     

In [6]:
customer_groupby = df.groupby(['customer_id'])['customer_id'].count()
print(customer_groupby.value_counts().sort_index())

1    99441
Name: customer_id, dtype: int64


In [7]:
customer__unique_groupby = df.groupby(['customer_unique_id'])['customer_unique_id'].count()
print(customer__unique_groupby.value_counts().sort_index())

1     93099
2      2745
3       203
4        30
5         8
6         6
7         3
9         1
17        1
Name: customer_unique_id, dtype: int64


On fait qu'il va faloir travailler avec les customer_unique_id

In [28]:
df_unique = customer["customer_unique_id"].unique()
df_unique.name('nb_commande)

SyntaxError: EOL while scanning string literal (30677980.py, line 2)

In [20]:
print(len(df_unique))

96096


In [24]:
df.merge(pd.DataFrame(data=df_unique, ), on="customer_unique_id", how ="inner")

KeyError: 'customer_unique_id'